<a href="https://colab.research.google.com/github/Ajay176854/Ajay/blob/main/Population_pred.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [3]:
df=pd.read_csv('/content/world_population.csv')

In [4]:
df.head()

,Rank,CCA3,Country/Territory,Capital,Continent,2022 Population,2020 Population,2015 Population,2010 Population,2000 Population,1990 Population,1980 Population,1970 Population,Area (km²),Density (per km²),Growth Rate,World Population Percentage
0,36,AFG,Afghanistan,Kabul,Asia,41128771,38972230,33753499,28189672,19542982,10694796,12486631,10752971,652230,63.0587,1.0257,0.52
1,138,ALB,Albania,Tirana,Europe,2842321,2866849,2882481,2913399,3182021,3295066,2941651,2324731,28748,98.8702,0.9957,0.04
2,34,DZA,Algeria,Algiers,Africa,44903225,43451666,39543154,35856344,30774621,25518074,18739378,13795915,2381741,18.8531,1.0164,0.56
3,213,ASM,American Samoa,Pago Pago,Oceania,44273,46189,51368,54849,58230,47818,32886,27075,199,222.4774,0.9831,0.00
4,203,AND,Andorra,Andorra la Vella,Europe,79824,77700,71746,71519,66097,53569,35611,19860,468,170.5641,1.0100,0.00


In [6]:
print(df.info())
print(df.describe())

target = '2022 Population'
features = [col for col in df.columns if col != target and 'Population' not in col]
population_features = [col for col in df.columns if 'Population' in col and col != target]
features.extend(population_features)

print(f"Target variable: {target}")
print(f"Features: {features}")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 234 entries, 0 to 233
Data columns (total 17 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Rank                         234 non-null    int64  
 1   CCA3                         234 non-null    object 
 2   Country/Territory            234 non-null    object 
 3   Capital                      234 non-null    object 
 4   Continent                    234 non-null    object 
 5   2022 Population              234 non-null    int64  
 6   2020 Population              234 non-null    int64  
 7   2015 Population              234 non-null    int64  
 8   2010 Population              234 non-null    int64  
 9   2000 Population              234 non-null    int64  
 10  1990 Population              234 non-null    int64  
 11  1980 Population              234 non-null    int64  
 12  1970 Population              234 non-null    int64  
 13  Area (km²)          

In [7]:
print(df.isnull().sum())

# Impute missing values in 'Growth Rate' and 'World Population Percentage' with the mean
df['Growth Rate'].fillna(df['Growth Rate'].mean(), inplace=True)
df['World Population Percentage'].fillna(df['World Population Percentage'].mean(), inplace=True)

print(df.isnull().sum())

Rank                           0
CCA3                           0
Country/Territory              0
Capital                        0
Continent                      0
2022 Population                0
2020 Population                0
2015 Population                0
2010 Population                0
2000 Population                0
1990 Population                0
1980 Population                0
1970 Population                0
Area (km²)                     0
Density (per km²)              0
Growth Rate                    0
World Population Percentage    0
dtype: int64
Rank                           0
CCA3                           0
Country/Territory              0
Capital                        0
Continent                      0
2022 Population                0
2020 Population                0
2015 Population                0
2010 Population                0
2000 Population                0
1990 Population                0
1980 Population                0
1970 Population               

/tmp/ipython-input-2466502083.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Growth Rate'].fillna(df['Growth Rate'].mean(), inplace=True)
/tmp/ipython-input-2466502083.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplac

In [10]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
import numpy as np

numerical_features = df.select_dtypes(include=np.number).columns.tolist()
categorical_features = df.select_dtypes(include='object').columns.tolist()

# Separate the features to be transformed from the rest
df_numerical = df[numerical_features]
df_categorical = df[categorical_features]
df_remaining = df.drop(columns=numerical_features + categorical_features)

# Scale numerical features
scaler = StandardScaler()
df_numerical_scaled = scaler.fit_transform(df_numerical)
df_numerical_scaled = pd.DataFrame(df_numerical_scaled, columns=numerical_features)

# Encode categorical features
encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
df_categorical_encoded = encoder.fit_transform(df_categorical)
categorical_feature_names = encoder.get_feature_names_out(categorical_features)
df_categorical_encoded = pd.DataFrame(df_categorical_encoded, columns=categorical_feature_names)

# Reset index for concatenation
df_remaining = df_remaining.reset_index(drop=True)
df_numerical_scaled = df_numerical_scaled.reset_index(drop=True)
df_categorical_encoded = df_categorical_encoded.reset_index(drop=True)


# Concatenate the processed dataframes
df_processed = pd.concat([df_numerical_scaled, df_categorical_encoded, df_remaining], axis=1)

display(df_processed.head())

,Rank,2022 Population,2020 Population,2015 Population,2010 Population,2000 Population,1990 Population,1980 Population,1970 Population,Area (km²),...,Capital_Yaounde,Capital_Yaren,Capital_Yerevan,Capital_Zagreb,Continent_Africa,Continent_Asia,Continent_Europe,Continent_North America,Continent_Oceania,Continent_South America
0,-1.206525,0.051690,0.040437,0.015551,-0.013356,-0.060349,-0.123080,-0.079622,-0.074411,0.040260,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,0.303482,-0.228850,-0.226417,-0.221689,-0.217275,-0.207138,-0.198879,-0.196580,-0.198997,-0.314379,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,-1.236133,0.079347,0.073545,0.060043,0.048495,0.040420,0.028762,-0.003005,-0.029431,1.024015,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,1.413781,-0.249353,-0.247265,-0.243445,-0.240337,-0.235164,-0.232142,-0.232222,-0.232961,-0.330618,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1.265741,-0.249092,-0.247032,-0.243289,-0.240202,-0.235094,-0.232083,-0.232189,-0.233068,-0.330465,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [11]:
from sklearn.model_selection import train_test_split

X = df_processed.drop('2022 Population', axis=1)
y = df_processed['2022 Population']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Shape of X_train:", X_train.shape)
print("Shape of X_test:", X_test.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of y_test:", y_test.shape)

Shape of X_train: (187, 720)
Shape of X_test: (47, 720)
Shape of y_train: (187,)
Shape of y_test: (47,)


In [12]:
from sklearn.linear_model import LinearRegression

# Choose the Linear Regression model
model = LinearRegression()

# Explain the choice of model
print("Chosen model: Linear Regression")
print("Explanation:")
print("- The problem is a regression task: predicting a continuous numerical variable (population).")
print("- Linear Regression is a simple and interpretable model that can serve as a good baseline.")
print("- The data has been preprocessed (scaled numerical features, one-hot encoded categorical features), which is suitable for Linear Regression.")
print("- The dataset size is relatively small, and Linear Regression is computationally efficient.")

Chosen model: Linear Regression
Explanation:
- The problem is a regression task: predicting a continuous numerical variable (population).
- Linear Regression is a simple and interpretable model that can serve as a good baseline.
- The data has been preprocessed (scaled numerical features, one-hot encoded categorical features), which is suitable for Linear Regression.
- The dataset size is relatively small, and Linear Regression is computationally efficient.


In [13]:
model.fit(X_train, y_train)

LinearRegression()

In [14]:
from sklearn.metrics import mean_squared_error, r2_score

# Make predictions on the testing data
y_pred = model.predict(X_test)

# Calculate Mean Squared Error
mse = mean_squared_error(y_test, y_pred)

# Calculate R-squared score
r2 = r2_score(y_test, y_pred)

# Print the evaluation metrics
print(f"Mean Squared Error (MSE): {mse}")
print(f"R-squared (R2) Score: {r2}")

Mean Squared Error (MSE): 0.0002977061990269458
R-squared (R2) Score: 0.9983033533642727


In [15]:
# Create a DataFrame for future data (example: predicting for 2025)
# This requires making assumptions about future values for each feature.
# For simplicity, let's create a future data point based on the last entry in the original df,
# assuming a plausible increase in population features and keeping others constant or making reasonable adjustments.

# Find the last row in the original dataframe before splitting
last_row_original_df = df.iloc[-1].copy()

# Create a future data point based on the last row
future_data = pd.DataFrame([last_row_original_df])

# Make some assumptions about future population growth (e.g., extrapolate from recent growth rate)
# This is a simplified approach and real-world prediction would involve more sophisticated forecasting.
future_data['2020 Population'] = future_data['2022 Population'] * (1 + future_data['Growth Rate'])**3 # Assuming prediction for 2025
future_data['2015 Population'] = future_data['2020 Population'] * (1 + future_data['Growth Rate'])**5 # This is just an example, real forecasting is complex
future_data['2010 Population'] = future_data['2015 Population'] * (1 + future_data['Growth Rate'])**5
future_data['2000 Population'] = future_data['2010 Population'] * (1 + future_data['Growth Rate'])**10
future_data['1990 Population'] = future_data['2000 Population'] * (1 + future_data['Growth Rate'])**10
future_data['1980 Population'] = future_data['1990 Population'] * (1 + future_data['Growth Rate'])**10
future_data['1970 Population'] = future_data['1980 Population'] * (1 + future_data['Growth Rate'])**10


# Preprocess the future data using the same preprocessing steps as the training data
# Scale numerical features
numerical_features = future_data.select_dtypes(include=np.number).columns.tolist()
categorical_features = future_data.select_dtypes(include='object').columns.tolist()

future_data_numerical = future_data[numerical_features]
future_data_categorical = future_data[categorical_features]


# Use the same scaler and encoder fitted on the training data
future_data_numerical_scaled = scaler.transform(future_data_numerical)
future_data_numerical_scaled = pd.DataFrame(future_data_numerical_scaled, columns=numerical_features)

future_data_categorical_encoded = encoder.transform(future_data_categorical)
future_data_categorical_encoded = pd.DataFrame(future_data_categorical_encoded, columns=encoder.get_feature_names_out(categorical_features))

# Concatenate the processed future dataframes
future_data_processed = pd.concat([future_data_numerical_scaled, future_data_categorical_encoded], axis=1)

# Ensure the columns of future_data_processed match X_train
missing_cols = set(X_train.columns) - set(future_data_processed.columns)
for c in missing_cols:
    future_data_processed[c] = 0
future_data_processed = future_data_processed[X_train.columns]


# Predict future population values
y_pred_future = model.predict(future_data_processed)

# Display the predicted future population values
print(f"Predicted future population: {y_pred_future[0]}")

Predicted future population: -163998398590660.9
